# Computed Attributes

The following creates a virtual attribute X that runs a calculation when fetched:

In [2]:
class AttrSquare:
    def __init__(self, start):
        self.value = start                  # Triggers __setattr__!
    def __getattr__(self, attr):            # On undefined attr fetch
        if attr == "X":
            return self.value ** 2          # value is not undefined
        else:
            raise AttributeError(attr)
    def __setattr__(self, attr, value):     # On all attr assignments
        if attr == 'X':
            attr = 'value'
        self.__dict__[attr] = value
        
A = AttrSquare(3)                           # 2 instance of class with overloading
B = AttrSquare(32)                          # Each has different state information

print(A.X)                      # 3 ** 2
A.X = 4
print(A.X)                      # 4 ** 2
print(B.X)

9
16
1024


Above script's mechanis are based on **generic attribute interception** methods.

### Using `__getattribute__`

Implementing the same above example using `__getattribute__` instead of `__getattr__`; the following replaces the fetch method with a `__getattribute__` and changes the `__setattr__` assignment method to avoid looping by using direct superclass method calls instead of `__dict__` keys:


In [4]:
class AttrSquare:                           # Add (object) for 2.X
    def __init__(self, start):
        self.value = start                  # Triggers __setattr__!
        
    def __getattribute__(self, attr):       # On all attr fetches
        if attr == "X":
            return self.value ** 2          # Triggers __getattributes__ again!
        else:
            return object.__getattribute__(self, attr)
    
    def __setattr__(self, attr, value):     # On all attr assignments
        if attr == 'X':
            attr = 'value'
        object.__setattr__(self, attr, value)
        
A = AttrSquare(3)                           # 2 instance of class with overloading
B = AttrSquare(32)                          # Each has different state information

print(A.X)                      # 3 ** 2
A.X = 4
print(A.X)                      # 4 ** 2
print(B.X)

9
16
1024


When this version, is run, the results are the same again. Notice, though,, the implicit routing gonig on inside the class's methods:

- **self.value=start** inside the constructor triggers `__setattr__`
- **self.value** inside `__getattribute__` triggers `__getattribute__` again.

In fact, the `__getattribute__` is run *twice* each time we fetch attribute **X**. This doesn't happen the `__getattr__` version, because the **value** attribute is not undefined. if you care about speed and want to avoid this, change `__getattribute__` to use the superclass to fetch **value** as well.

In [ ]:
def __getattribute__(self, attr):
    if attr == "X":
        return object.__getattribute__(self, 'value') ** 2

Of cource, this still incurs a call to the superclass method, but not an additional recursive call before we get there. Add **print** calls to these methods to trace how and when they run.